# 🤖 步骤 2：LLM 概念提取

这个 notebook 负责：
1. 使用 OpenAI GPT 从文本中提取概念
2. 识别概念之间的关系
3. 评估概念重要性
4. 保存提取结果

**⚠️ 注意**：此步骤会调用 OpenAI API，产生费用！
**⏱️ 预计时间**：取决于块数量，30块约需 5-10 分钟
**💰 预计费用**：30块约 $1-3（使用 gpt-3.5-turbo）

## 1. 加载环境和配置

In [ ]:
import os
import sys
import json
import yaml
import logging
from typing import Dict, List, Tuple
from datetime import datetime

import pandas as pd
from tqdm import tqdm
import openai

# 加载环境变量
if os.path.exists('/content/PWD_Project/.env'):
    with open('/content/PWD_Project/.env', 'r') as f:
        for line in f:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                os.environ[key] = value

PROJECT_ROOT = os.environ.get('PROJECT_ROOT', '/content/PWD_Project')
CONFIG_PATH = os.environ.get('CONFIG_PATH', f'{PROJECT_ROOT}/config/config_colab.yaml')

os.chdir(PROJECT_ROOT)
sys.path.insert(0, f'{PROJECT_ROOT}/modules')

# 加载配置
with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
    CONFIG = yaml.safe_load(f)

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('ConceptExtractor')

print(f"✅ 配置已加载")
print(f"   LLM 模型: {CONFIG['llm']['model']}")
print(f"   最大处理块数: {CONFIG['llm']['max_chunks']}")

## 2. 加载上一步的文本块

In [ ]:
# 加载文本块
output_dir = CONFIG['output']['base_directory']
chunks_file = f"{output_dir}/text_chunks.json"

if not os.path.exists(chunks_file):
    print(f"❌ 找不到文本块文件: {chunks_file}")
    print("   请先运行 01_PDF提取.ipynb")
    raise FileNotFoundError(chunks_file)

with open(chunks_file, 'r', encoding='utf-8') as f:
    chunks = json.load(f)

print(f"✅ 加载了 {len(chunks)} 个文本块")
print(f"\n⚠️  将处理前 {CONFIG['llm']['max_chunks']} 个块（配置限制）")

## 3. 概念提取器（OpenAI 版本）

In [ ]:
class ConceptExtractorOpenAI:
    """使用 OpenAI API 进行概念提取"""
    
    def __init__(self, api_key: str, model: str = 'gpt-3.5-turbo', 
                 temperature: float = 0.1, timeout: int = 120):
        self.api_key = api_key
        self.model = model
        self.temperature = temperature
        self.timeout = timeout
        openai.api_key = api_key
        
        # 统计信息
        self.stats = {
            'total_requests': 0,
            'successful_requests': 0,
            'failed_requests': 0,
            'total_concepts': 0,
            'total_relationships': 0
        }
    
    def extract_from_chunks(self, chunks: List[Dict], max_chunks: int = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """从文本块批量提取概念和关系"""
        if max_chunks:
            chunks = chunks[:max_chunks]
            logger.info(f"限制处理 {max_chunks} 个文本块")
        
        all_concepts = []
        all_relationships = []
        
        for chunk in tqdm(chunks, desc="🤖 LLM 概念提取"):
            try:
                result = self._extract_from_text(chunk['text'], chunk['chunk_id'])
                
                # 处理概念
                for concept in result.get('concepts', []):
                    all_concepts.append({
                        'entity': concept['name'],
                        'category': concept.get('category', 'unknown'),
                        'importance': concept.get('importance', 1),
                        'chunk_id': chunk['chunk_id'],
                        'source_pdf': chunk['source_pdf']
                    })
                
                # 处理关系
                for relation in result.get('relationships', []):
                    all_relationships.append({
                        'node_1': relation['head'],
                        'node_2': relation['tail'],
                        'edge': relation['relation'],
                        'weight': relation.get('confidence', 0.8),
                        'source': 'llm',
                        'chunk_id': chunk['chunk_id']
                    })
                
                self.stats['successful_requests'] += 1
                self.stats['total_concepts'] += len(result.get('concepts', []))
                self.stats['total_relationships'] += len(result.get('relationships', []))
                
            except Exception as e:
                logger.error(f"处理块 {chunk['chunk_id']} 失败: {e}")
                self.stats['failed_requests'] += 1
                continue
            
            self.stats['total_requests'] += 1
        
        logger.info(f"提取完成: {len(all_concepts)} 个概念, {len(all_relationships)} 个关系")
        
        return pd.DataFrame(all_concepts), pd.DataFrame(all_relationships)
    
    def _extract_from_text(self, text: str, chunk_id: str) -> Dict:
        """使用 GPT 从单个文本块提取概念"""
        # 限制文本长度（控制成本）
        text = text[:1500]
        
        system_prompt = """你是一个专业的生物学知识提取助手，专注于松材线虫病领域。
请从给定文本中提取关键概念和它们之间的关系。

概念类别包括：
- 病原体（如：松材线虫）
- 宿主（如：松树、黑松）
- 载体（如：松墨天牛）
- 症状（如：针叶变色、树脂减少）
- 防治方法（如：化学防治、生物防治）
- 地区（如：日本、中国）
- 其他相关概念

请只返回 JSON 格式，不要任何其他文字。"""
        
        user_prompt = f"""文本：
{text}

请提取概念和关系，返回 JSON：
{{
  "concepts": [
    {{"name": "概念名称", "category": "类别", "importance": 1-3}}
  ],
  "relationships": [
    {{"head": "概念1", "tail": "概念2", "relation": "关系类型", "confidence": 0.0-1.0}}
  ]
}}"""
        
        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=self.temperature,
                max_tokens=800,
                timeout=self.timeout
            )
            
            content = response.choices[0].message.content.strip()
            
            # 尝试提取 JSON（可能在代码块中）
            if '```json' in content:
                content = content.split('```json')[1].split('```')[0].strip()
            elif '```' in content:
                content = content.split('```')[1].split('```')[0].strip()
            
            result = json.loads(content)
            return result
            
        except json.JSONDecodeError as e:
            logger.warning(f"JSON 解析失败 ({chunk_id}): {e}")
            logger.debug(f"原始响应: {content[:200]}...")
            return {'concepts': [], 'relationships': []}
        except Exception as e:
            logger.error(f"API 调用失败 ({chunk_id}): {e}")
            return {'concepts': [], 'relationships': []}
    
    def get_stats(self) -> Dict:
        """获取统计信息"""
        return self.stats

print("✅ ConceptExtractorOpenAI 类定义完成")

## 4. 执行概念提取

**⚠️ 重要提示**：
- 此步骤会消耗 OpenAI API 配额
- 处理 30 个块预计费用 $1-3
- 可随时中断，已处理的数据会保留

In [ ]:
# 验证 API Key
if not CONFIG['llm']['api_key']:
    print("❌ OpenAI API Key 未配置！")
    print("   请返回 00_环境设置.ipynb 配置 API Key")
    raise ValueError("Missing OpenAI API Key")

# 创建提取器
extractor = ConceptExtractorOpenAI(
    api_key=CONFIG['llm']['api_key'],
    model=CONFIG['llm']['model'],
    temperature=CONFIG['llm']['temperature'],
    timeout=CONFIG['llm']['timeout']
)

print(f"\n✅ 提取器已初始化")
print(f"   模型: {CONFIG['llm']['model']}")
print(f"   温度: {CONFIG['llm']['temperature']}")
print(f"\n⚠️  即将处理 {min(len(chunks), CONFIG['llm']['max_chunks'])} 个文本块")
print(f"   预计时间: 5-10 分钟")
print(f"   预计费用: $1-3 (使用 gpt-3.5-turbo)")
print("\n按 Ctrl+C 可随时中断")

In [ ]:
# 执行提取
print("\n" + "="*60)
print("开始概念提取")
print("="*60 + "\n")

start_time = datetime.now()

try:
    concepts_df, relationships_df = extractor.extract_from_chunks(
        chunks,
        max_chunks=CONFIG['llm']['max_chunks']
    )
    
    end_time = datetime.now()
    duration = end_time - start_time
    
    print("\n" + "="*60)
    print("概念提取完成")
    print("="*60)
    print(f"\n耗时: {duration}")
    
    # 显示统计
    stats = extractor.get_stats()
    print(f"\n📊 统计信息:")
    print(f"   总请求数: {stats['total_requests']}")
    print(f"   成功: {stats['successful_requests']}")
    print(f"   失败: {stats['failed_requests']}")
    print(f"   提取的概念: {stats['total_concepts']}")
    print(f"   提取的关系: {stats['total_relationships']}")
    
except KeyboardInterrupt:
    print("\n⚠️  用户中断，保存已处理的数据...")
    concepts_df = pd.DataFrame([])
    relationships_df = pd.DataFrame([])
except Exception as e:
    print(f"\n❌ 提取过程出错: {e}")
    raise

## 5. 查看提取结果

In [ ]:
if not concepts_df.empty:
    print("\n📊 概念样例（前10个）:")
    print(concepts_df.head(10))
    
    print("\n📊 概念类别分布:")
    print(concepts_df['category'].value_counts())
    
    print("\n📊 重要性分布:")
    print(concepts_df['importance'].value_counts().sort_index())
else:
    print("⚠️  未提取到概念")

In [ ]:
if not relationships_df.empty:
    print("\n📊 关系样例（前10个）:")
    print(relationships_df.head(10))
    
    print("\n📊 关系类型分布:")
    print(relationships_df['edge'].value_counts().head(10))
else:
    print("⚠️  未提取到关系")

## 6. 保存结果

In [ ]:
# 保存概念和关系
concepts_file = f"{output_dir}/concepts_raw.csv"
relationships_file = f"{output_dir}/relationships_raw.csv"

if not concepts_df.empty:
    concepts_df.to_csv(concepts_file, index=False, encoding='utf-8-sig')
    print(f"✅ 概念已保存: {concepts_file}")

if not relationships_df.empty:
    relationships_df.to_csv(relationships_file, index=False, encoding='utf-8-sig')
    print(f"✅ 关系已保存: {relationships_file}")

# 保存统计信息
extraction_stats = {
    'timestamp': datetime.now().isoformat(),
    'duration': str(duration),
    'llm_model': CONFIG['llm']['model'],
    'chunks_processed': min(len(chunks), CONFIG['llm']['max_chunks']),
    'api_stats': stats,
    'concepts_extracted': len(concepts_df),
    'relationships_extracted': len(relationships_df)
}

stats_file = f"{output_dir}/extraction_llm_stats.json"
with open(stats_file, 'w', encoding='utf-8') as f:
    json.dump(extraction_stats, f, ensure_ascii=False, indent=2)

print(f"✅ 统计信息已保存: {stats_file}")

## 7. 概念提取完成

### ✅ 完成情况

- 处理的文本块数量
- 提取的概念数量
- 提取的关系数量
- API 调用统计

### 📝 下一步

请打开 **`03_概念去重.ipynb`** 继续执行概念去重

### 💡 提示

- 原始提取结果已保存，后续步骤不会重复调用 API
- 如需重新提取，删除输出文件后重新运行即可
- 可以调整 `max_chunks` 参数处理更多或更少的文本

In [ ]:
# 显示完成总结
print("\n" + "="*60)
print("🎉 概念提取步骤完成！")
print("="*60)
print(f"\n📊 提取结果:")
print(f"   - 处理块数: {extraction_stats['chunks_processed']}")
print(f"   - 概念数: {extraction_stats['concepts_extracted']}")
print(f"   - 关系数: {extraction_stats['relationships_extracted']}")
print(f"   - 耗时: {extraction_stats['duration']}")
print(f"\n📁 文件保存在: {output_dir}")
print(f"\n➡️  下一步: 打开 03_概念去重.ipynb")